### **Introduction**

This project focuses on identifying high-value customers and churn risks within the retail industry. Using an RFM (Recency, Frequency, Monetary) framework, I segmented a database of 4,000+ customers to drive data-informed marketing strategies and deliver actionable business insights.

#### **Business Questions to be Answered**
1. Who are our most valuable customers? (Focus: Identifying the "Champions" segment that drives the core revenue.)

2. Which segments exhibit critically high Recency (time since the last purchase), indicating a churn risk that requires an immediate reactivation campaign? (Focus: Pinpointing "At Risk" and "Almost Lost" segments for urgent intervention.)

3. Which customers purchase frequently but have a low Monetary value, and what strategies can be used to incentivize them to increase their Average Order Value (AOV)? (Focus: Understanding the "Loyal" vs. "High Frequency" dynamics to drive upselling.)

4. What is the Average Order Value (AOV) across different customer segments? (Focus: Benchmarking transaction efficiency to identify which segments are the most profitable per visit.)

#### **Project Assumptions**

The analysis is based on a snapshot of historical data up to the latest transaction date in the dataset.

Returns and cancelled orders were removed during the data cleaning phase to ensure accuracy in Monetary and Frequency metrics.

Segmentation is strictly behavioral (RFM) and does not include demographic variables.

#### **1. Data Import and Exploration**

In this initial phase, we load the dataset and perform an Exploratory Data Analysis (EDA). This step is crucial to understanding the data structure, identifying the total number of records, inspecting column types, and detecting missing values or anomalies that could affect the RFM calculation.

In [1]:
import pandas as pd

import numpy as np

In [5]:
df = pd.read_csv('online_retail.csv')

df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [8]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


#### **2. Data Cleaning**

In this stage, we identify and handle missing values that could compromise the integrity of the RFM analysis. Additionally, we address anomalies such as cancelled orders or returns to ensure our Monetary and Frequency metrics are accurate.

**Professional Best Practice**: We will create a working copy of the raw dataset to maintain the original data while performing transformations.

In [9]:
df_clean = df.copy(deep = True)

In [10]:
df_clean.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


After inspecting the dataset, we can observe that only two columns contain null values: 'Description' and 'CustomerID'.

Among these, 'CustomerID' is critical for our analysis. Since this ID serves as a unique identifier, it is impossible to impute missing values using statistical methods like the mean or median. Without this data, we cannot track individual customer behavior or assign them to specific segments for targeted marketing.

To maintain the integrity of the RFM model, rows with missing 'CustomerID' values will be removed, as they would otherwise negatively impact the accuracy of our segmentation.

In [12]:
# # Dropping rows with missing CustomerID to ensure data integrity
df_clean.dropna(
    subset = ['CustomerID'],
    inplace= True)

# Final check to confirm a clean dataset without null values
df_clean.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,0
Country,0


In [13]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


As observed, 135,080 records were removed, representing approximately 24.9% of the original dataframe.

**Note**: The significant reduction in dataset size is primarily due to the removal of guest checkouts or transactions missing a unique CustomerID, which are unsuitable for individual behavioral tracking in an RFM model.

Next, we will identify and remove all duplicate rows from the dataset. This step is essential to ensure that our Frequency metric accurately reflects unique transactions and prevents any artificial inflation of sales data.

In [14]:
df_clean.drop_duplicates(inplace = True)

df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 401604 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    401604 non-null  object 
 1   StockCode    401604 non-null  object 
 2   Description  401604 non-null  object 
 3   Quantity     401604 non-null  int64  
 4   InvoiceDate  401604 non-null  object 
 5   UnitPrice    401604 non-null  float64
 6   CustomerID   401604 non-null  float64
 7   Country      401604 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.6+ MB


As shown, there were 5,225 duplicate records, representing approximately 0.96% of the original dataframe.

Additionally, we will filter the 'Quantity' column to retain only positive values. In retail datasets, negative quantities typically represent returns or cancellations. For the purpose of this analysis, we will focus exclusively on successful sales to establish a baseline for customer purchase behavior.

**Note**: Removing returns ensures that the Frequency metric reflects actual purchase intent and that the Monetary value represents realized revenue, avoiding distortions from refunded transactions.

In [15]:
df_clean = df_clean[df_clean['Quantity'] > 0].copy()

In [16]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392732 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    392732 non-null  object 
 1   StockCode    392732 non-null  object 
 2   Description  392732 non-null  object 
 3   Quantity     392732 non-null  int64  
 4   InvoiceDate  392732 non-null  object 
 5   UnitPrice    392732 non-null  float64
 6   CustomerID   392732 non-null  float64
 7   Country      392732 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.0+ MB


As shown above, 8,872 records were filtered out, representing approximately 1.63% of the original dataframe.

At this stage, our dataset is free of null values, duplicates, and inconsistent transaction records. Having refined our key columns, we are now ready to proceed with data standardization and data transformation.

#### **3. Data Transformation**

Now that our dataset is clean, we can proceed with data transformation. This stage involves adjusting data types, creating new calculated columns, and aggregating the data to prepare it for the final analysis.

One of the most critical columns for this analysis is 'InvoiceDate', which must be in 'datetime' format to perform time based calculations. We will now proceed with this transformation to ensure we can accurately calculate the number of days since a customer's last purchase.

In [17]:
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

In [27]:
print(df_clean['InvoiceDate'].min())
print(df_clean['InvoiceDate'].max())

2010-12-01 08:26:00
2011-12-09 12:50:00


We need to create a new column representing the total value of each transaction. This column is calculated by multiplying 'Quantity' and 'UnitPrice'. This derived metric will be crucial for determining the Monetary (M) value within our RFM framework, allowing us to quantify the total revenue generated by each customer.

In [28]:
df_clean['Total'] = df_clean['Quantity'] * df_clean['UnitPrice']

In [29]:
df_clean.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom,15.30
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,25.50
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,11.10
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,11.10
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,54.08


#### **4. RFM Calculation**

In this section, we aggregate our transaction data at the customer level to calculate the three pillars of the RFM framework:

**Recency (R)**: Measures how recently a customer made a purchase. We establish a reference date (a simulated "Today's Date") and calculate the difference in days between this date and each customer's last transaction.

**Frequency (F)**: Measures how often a customer has purchased. We count the number of unique transactions (InvoiceNo) associated with each CustomerID.

**Monetary Value (M)**: Measures the total amount of money a customer has spent. We calculate the sum of the 'TotalSales' column for each CustomerID.

In [30]:
# 1. Define a reference date to calculate Recency.
# We use the maximum invoice date plus one day to simulate "today."
Now = df_clean['InvoiceDate'].max() + pd.Timedelta(days=1)

# 2. Group the data by 'CustomerID' to calculate raw Recency, Frequency, and Monetary values.
# Frequency is calculated as the count of unique invoice numbers.
df_rfm = df_clean.groupby('CustomerID').agg(
    Recency = ('InvoiceDate', 'max'),
    Frequency = ('InvoiceNo', 'nunique'),
    Monetary = ('Total', 'sum')
)

# 3. Calculate actual Recency as the number of days between "Now" and the last purchase date.
df_rfm['Recency'] = (Now - df_rfm['Recency']).dt.days

# 4. Reset the index to turn 'CustomerID' back into a standard column.
df_rfm = df_rfm.reset_index()


df_rfm.head(10)


,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,1,77183.60
1,12347.0,2,7,4310.00
2,12348.0,75,4,1797.24
3,12349.0,19,1,1757.55
4,12350.0,310,1,334.40
5,12352.0,36,8,2506.04
6,12353.0,204,1,89.00
7,12354.0,232,1,1079.40
8,12355.0,214,1,459.40
9,12356.0,23,3,2811.43


#### **5. RFM Scoring**

Now that we have the raw RFM data, we can calculate the scores for each customer to segment them for further analysis. We must transform the raw metrics into a standardized scale (from 1 to 5). This process is known as Scoring.

**Important Logic for the Model**

**Recency Score**: Unlike the other metrics, a lower raw Recency value (fewer days since the last purchase) is better. Therefore, customers with the lowest Recency get the highest score (5).

**Frequency & Monetary Scores**: For these metrics, higher raw values are better. Customers who buy more often and spend more receive the highest score (5).

To ensure a balanced distribution, we use Quintiles (dividing the population into five equal groups of 20%).

In [31]:
# Recency (5-1): A score of 5 is assigned to the lowest recency values (customers who purchased most recently).
# Frequency (1-5): A score of 5 is assigned to the highest frequency values (customers who purchased most often).
# Monetary (1-5): A score of 5 is assigned to the highest monetary values (customers who spent the most money).

df_rfm['R_Score'] = pd.qcut(
    df_rfm['Recency'],
    q = 5,
    labels = ['5','4','3','2','1'],
    duplicates = 'drop'
)


df_rfm['M_Score'] = pd.qcut(
    df_rfm['Monetary'],
    q = 5,
    labels = ['1','2','3','4','5'],
    duplicates = 'drop'
)

df_rfm.head(10)

,CustomerID,Recency,Frequency,Monetary,R_Score,M_Score
0,12346.0,326,1,77183.60,1,5
1,12347.0,2,7,4310.00,5,5
2,12348.0,75,4,1797.24,2,4
3,12349.0,19,1,1757.55,4,4
4,12350.0,310,1,334.40,1,2
5,12352.0,36,8,2506.04,3,5
6,12353.0,204,1,89.00,1,1
7,12354.0,232,1,1079.40,1,4
8,12355.0,214,1,459.40,1,2
9,12356.0,23,3,2811.43,4,5


**Technical Note**:
During the scoring process, we observed that the Frequency distribution was heavily skewed toward single purchase customers. Using standard automated quintiles would result in overlapping bins.

To ensure a fair and accurate classification, we calculated the specific quantile thresholds and applied manual bins using pd.cut(). This allows us to clearly distinguish between one time buyers, occasional shoppers, and our most frequent customers.

In [32]:
# 1. Defining the quantiles to inspect the distribution
quantile_list = [0.2, 0.4, 0.6, 0.8]
cut_points = df_rfm['Frequency'].quantile(quantile_list).to_list()

print("The number of quantiles: ", cut_points)

# 2. Defining manual bins based on the distribution
# We use 0.9 as the lower bound to capture 1.0 in the first bin.
# We add +1 to the maximum value to ensure the last bin is inclusive.
bins = [0.9, 1.0, 2.0, 3.0, 6.0, df_rfm['Frequency'].max() + 1]

# 3. Applying pd.cut() to assign the Frequency Score (F_Score)
df_rfm['F_Score'] = pd.cut(
    df_rfm['Frequency'],
    bins=bins,
    labels=['1','2','3','4','5'], # 5 represents the highest frequency
    include_lowest=True
)

df_rfm.head(10)

The number of quantiles:  [1.0, 2.0, 3.0, 6.0]


,CustomerID,Recency,Frequency,Monetary,R_Score,M_Score,F_Score
0,12346.0,326,1,77183.60,1,5,1
1,12347.0,2,7,4310.00,5,5,5
2,12348.0,75,4,1797.24,2,4,4
3,12349.0,19,1,1757.55,4,4,1
4,12350.0,310,1,334.40,1,2,1
5,12352.0,36,8,2506.04,3,5,5
6,12353.0,204,1,89.00,1,1,1
7,12354.0,232,1,1079.40,1,4,1
8,12355.0,214,1,459.40,1,2,1
9,12356.0,23,3,2811.43,4,5,3


We have reached a pivotal step: creating the RFM Segmentation column. This consists of concatenating the R_Score, F_Score, and M_score into a single string.

This combined score acts as a unique behavioral signature for each customer. Each specific combination (e.g., '555' or '111') allows us to categorize our database into distinct groups for targeted marketing strategies.


In [33]:
# We convert the scores to strings and concatenate them to create an unique RFM ID.
df_rfm['R_Score'] = df_rfm['R_Score'].astype(str)
df_rfm['F_Score'] = df_rfm['F_Score'].astype(str)
df_rfm['M_Score'] = df_rfm['M_Score'].astype(str)

df_rfm['RFM_Segment'] = df_rfm['R_Score'] + df_rfm['F_Score'] + df_rfm['M_Score']

df_rfm.head(10)

,CustomerID,Recency,Frequency,Monetary,R_Score,M_Score,F_Score,RFM_Segment
0,12346.0,326,1,77183.60,1,5,1,115
1,12347.0,2,7,4310.00,5,5,5,555
2,12348.0,75,4,1797.24,2,4,4,244
3,12349.0,19,1,1757.55,4,4,1,414
4,12350.0,310,1,334.40,1,2,1,112
5,12352.0,36,8,2506.04,3,5,5,355
6,12353.0,204,1,89.00,1,1,1,111
7,12354.0,232,1,1079.40,1,4,1,114
8,12355.0,214,1,459.40,1,2,1,112
9,12356.0,23,3,2811.43,4,5,3,435


Managing 125 unique RFM combinations is impractical for business strategy. To ensure the analysis provides clear, actionable insights and effective data visualizations, we have consolidated these scores into 8 descriptive segments (e.g., "New Customers").

A 9th category, "Others," will be used for remaining combinations. This simplification is designed to focus on the most critical customer behaviors and streamline future decision-making processes.

In [34]:
df_rfm['R'] = df_rfm['R_Score'].astype(int)
df_rfm['F'] = df_rfm['F_Score'].astype(int)
df_rfm['M'] = df_rfm['M_Score'].astype(int)

conditions = [
    # Champions, most valuable customers.They spend the most money, often, and frequently.
    (df_rfm['R'] == 5) & (df_rfm['F'] == 5) & (df_rfm['M'] == 5),

    # New Customers, with the lowest recency and non-frequent.
    (df_rfm['R'] == 5) & (df_rfm['F'] == 1),

    # At Risk, spend good money, and purchased often, but long time ago.
    (df_rfm['R'] <= 2) & (df_rfm['F'] >= 4) & (df_rfm['M'] >= 4),

    # Loyal, spend good money, but not that often as the champions.
    (df_rfm['R'] >= 4) & (df_rfm['F'] >= 4) & (df_rfm['M'] >= 4),

    # Potential Loyal, they spend more than average, but not that often.
    (df_rfm['R'] >= 3) & (df_rfm['F'] >= 3) & (df_rfm['M'] >= 4),

    # Almost Lost, bought long time ago, low money value, and low frequency.
    (df_rfm['R'] == 1) & (df_rfm['F'] == 2) & (df_rfm['M'] <= 3),

    # Lost, only bought once and long time ago.
    (df_rfm['R'] == 1) & (df_rfm['F'] == 1),

    # Low Commitment, bellow average recency.
    (df_rfm['R'] <= 2)
]

segments_names = [
    "Champions",
    "New Customers",
    "At Risk",
    "Loyal",
    "Potential Loyal",
    "Almost Lost",
    "Lost",
    "Low Commitment"
]

df_rfm['Segment_Name'] = np.select(
    conditions,
    segments_names,
    default = "Others"
)

df_rfm.head(10)

,CustomerID,Recency,Frequency,Monetary,R_Score,M_Score,F_Score,RFM_Segment,R,F,M,Segment_Name
0,12346.0,326,1,77183.60,1,5,1,115,1,1,5,Lost
1,12347.0,2,7,4310.00,5,5,5,555,5,5,5,Champions
2,12348.0,75,4,1797.24,2,4,4,244,2,4,4,At Risk
3,12349.0,19,1,1757.55,4,4,1,414,4,1,4,Others
4,12350.0,310,1,334.40,1,2,1,112,1,1,2,Lost
5,12352.0,36,8,2506.04,3,5,5,355,3,5,5,Potential Loyal
6,12353.0,204,1,89.00,1,1,1,111,1,1,1,Lost
7,12354.0,232,1,1079.40,1,4,1,114,1,1,4,Lost
8,12355.0,214,1,459.40,1,2,1,112,1,1,2,Lost
9,12356.0,23,3,2811.43,4,5,3,435,4,3,5,Potential Loyal


#### **6. Data Export & Final Output**

With this final transformation, our dataset now contains all the necessary metrics and segments for advanced data visualization.

We will export the results into a .csv file, which can be seamlessly integrated into our preferred Business Intelligence tool (Tableau in this case) to create interactive dashboards and uncover deeper business insights


In [ ]:
df_rfm.to_csv('rfm_v1.csv', index = False)